In [ ]:
Daum에서 테넷의 리뷰를 수집하여 네이버리뷰로 학습시킨 결과를 가져와 
분류가 얼마나 잘 되는지 확인
어느 브라우저의 리뷰가 더 긍정인지, 부정인지 확인

In [40]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from pandas import Series, DataFrame
import pandas as pd

In [ ]:
# 셀레리움으로 수집


url = 'https://movie.daum.net/moviedb/grade?movieId=132861'

driver = webdriver.Chrome('c:/data/chromedriver.exe')
driver.get(url)       # 웹브라우저 열기

In [ ]:
# 아래로 내려 과거의 리뷰를 모두 띄우기
for i in range(1,108):
    driver.find_element_by_xpath('//*[@id="alex-area"]/div/div/div/div[3]/div[1]/button').click()
    time.sleep(3)
    i = i+ 1

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
driver.close()

In [ ]:
# 리뷰가 들어있는 부분만 이용
li = soup.select('body > div.kakao_wrap.detail_type.head_fixed > main > article > div.section_detail > div.box_detailinfo > div.detail_cmtinfo > div.cmt_netizen > div#alex-area > div.pc > div.alex_single > div.wrap_alex > div.cmt_box > ul.list_comment > li')

In [ ]:
# 점수와 리뷰를 수집

daum_score = []
daum_review = []
for i in range(1,1030):
    daum_score.append(int(soup.select('div.cmt_info > div.ratings')[i].text))
    daum_review.append(soup.select('div.cmt_info >  p')[i].text)

daum_movie = DataFrame({'score': daum_score, 'review': daum_review})

In [ ]:
# review 값이 없는 행 제거  -> 1029개
daum_movie = daum_movie.drop(daum_movie[daum_movie.review == ''].index)

In [ ]:
# 긍부정으로 나누기

feeling2 = []
for i in daum_movie.score:
    if i < 5:
        feeling2.append('부정')
    elif i < 8 and i > 4:
        feeling2.append('중립')
    elif i >7:
        feeling2.append('긍정')

In [ ]:
len(feeling2)

In [ ]:
daum_movie['feeling'] = feeling2
daum_movie

In [4]:
import pickle

In [ ]:
# 바이너리 파일로 저장
with open('c:/data/daum_movie.pkl', 'wb') as file:
    pickle.dump(daum_movie, file)

In [8]:
# 바이너리파일 불러오기
with open('c:/data/daum_movie.pkl', 'rb') as file:
    daum_movie = pickle.load(file)

In [68]:
daum_movie

,score,review,feeling
0,10,나만 당할순 없다,긍정
1,1,유명해져라 그럼 우매한 대중은 니가 똥을싸도 박수를 쳐줄것이다,부정
2,3,대체 뭔 내용인지 뭘 말하려고 하는지 당최 모르겄다\n놀란 감독의 골수팬이지만 테넷...,부정
3,6,과거로 가서 췌장암 조기 발견하면 끝인데 이렇게 복잡하게 할 필요가 있었나?,중립
4,6,어렵다,중립
...,...,...,...
1024,10,평점 알바들 많네;;,긍정
1025,10,곳곳에 남겨진 흔적,긍정
1026,5,인셉션 인터스텔라 기대하고 보면 실망\n한번보고 재밌다고 하면 허세충. \n내용 너...,중립
1027,10,인셉션 덩케르크 다크나이트 다 재밌게봤는데 이건 진짜 재미없다,긍정


In [9]:
# 점수별 리뷰 수
daum_movie.review.groupby(daum_movie.score).count()

score
0      40
1     109
2      36
3      43
4      31
5      49
6      72
7      77
8      95
9     100
10    377
Name: review, dtype: int64

In [10]:
# 긍정 부정 중립의 수 확인
daum_movie.review.groupby(daum_movie.feeling).count()

feeling
긍정    572
부정    259
중립    198
Name: review, dtype: int64

In [13]:
# 전처리
import re

review_clean2 = [re.sub('[@#$%&*-?“”\.\,\'\"\(\)]+', '', i) for i in daum_movie.review]
review_clean2 = [re.sub('\d+.', '', i) for i in review_clean2]
review_clean2 = [re.sub('ost|OST|Ost|BGM|bgm', '음향', i) for i in review_clean2]
review_clean2 = [re.sub('TENET|tenet', '테넷', i) for i in review_clean2]
review_clean2 = [re.sub('cgv|CGV', '영화관', i) for i in review_clean2]
review_clean2 = [re.sub('[a-zA-Z]+', '', i) for i in review_clean2]

In [14]:
# 형태소 테그과정

from konlpy.tag import Okt
okt = Okt()

In [15]:
def okt_pos(arg):
    token = []
    for i in okt.pos(arg):
        if i[1] in ['Noun', 'Verb', 'Adjective', 'Punctuation']:
            token.append(i[0])
    return list(token)

In [16]:
token2 = [okt_pos(i) for i in review_clean2]

In [17]:
token2

[['나', '당할', '순', '없다'],
 ['유명해져라', '그럼', '우매', '대중', '니', '똥', '싸도', '박수', '쳐줄것이다'],
 ['대체',
  '내용',
  '뭘',
  '말',
  '하려고',
  '하는지',
  '최',
  '모르겄다',
  '놀란',
  '감독',
  '골수팬',
  '테넷',
  '옹호',
  '할',
  '래야',
  '할',
  '수가',
  '없네',
  '뭘',
  '알아야',
  '옹호',
  '해도',
  '하지',
  '배우',
  '찍고',
  '나서도',
  '영화',
  '내용',
  '이해',
  '했다는',
  '말',
  '들었지만',
  '영화',
  '일반',
  '대중',
  '알',
  '수',
  '있도록',
  '편집',
  '연출',
  '할',
  '줄',
  '알았는데',
  '이번',
  '관객',
  '대한',
  '배려',
  '입장',
  '전혀',
  '고려',
  '하지',
  '않은',
  '채',
  '본인',
  '세계',
  '구축',
  '하는데만',
  '힘',
  '쓴',
  '듯',
  '하다',
  '한',
  '가지',
  '의문',
  '드는',
  '건',
  '놀란',
  '감독',
  '본인',
  '이',
  '내용',
  '이해',
  '연출',
  '건지',
  '아님',
  '본인',
  '조차',
  '이해',
  '하지',
  '못',
  '채',
  '있어',
  '보이는',
  '영화',
  '만들려고',
  '건',
  '그게',
  '궁금하다'],
 ['과거', '가서', '췌장암', '조기', '발견', '하면', '끝', '복잡하게', '할', '필요', '있었나'],
 ['어렵다'],
 ['상상', '하기', '힘든것을', '표현', '해내는', '감독', '위대하다', '대중', '이해', '하기엔', '어렵다'],
 ['터미네이터',
  '존',
  '코너',
  '인터스텔라',
  '쿠퍼',
  '백투더퓨처

In [18]:
contents_token2 = [' '.join(i) for i in token2]
contents_token2

['나 당할 순 없다',
 '유명해져라 그럼 우매 대중 니 똥 싸도 박수 쳐줄것이다',
 '대체 내용 뭘 말 하려고 하는지 최 모르겄다 놀란 감독 골수팬 테넷 옹호 할 래야 할 수가 없네 뭘 알아야 옹호 해도 하지 배우 찍고 나서도 영화 내용 이해 했다는 말 들었지만 영화 일반 대중 알 수 있도록 편집 연출 할 줄 알았는데 이번 관객 대한 배려 입장 전혀 고려 하지 않은 채 본인 세계 구축 하는데만 힘 쓴 듯 하다 한 가지 의문 드는 건 놀란 감독 본인 이 내용 이해 연출 건지 아님 본인 조차 이해 하지 못 채 있어 보이는 영화 만들려고 건 그게 궁금하다',
 '과거 가서 췌장암 조기 발견 하면 끝 복잡하게 할 필요 있었나',
 '어렵다',
 '상상 하기 힘든것을 표현 해내는 감독 위대하다 대중 이해 하기엔 어렵다',
 '터미네이터 존 코너 인터스텔라 쿠퍼 백투더퓨처 역 재생 재생 효과 여러 내용 섞여 있는 느낌 듭니 화면 전환 다소 빨라 따라가기 조금 힘 듭니 번 보고 정확히 알기 어렵네요',
 '평점 왜 이럼 하도 사람 이해 안된다고 하길래 메멘토 처음 봤을 때 그 느낌 정도 되나 하는 마음 봤는데 시간 동안 시간 가는 줄 모르고 재밌게 봤음 어려운 용어 개념 닐 중 캐릭 얘기 해주더만 다만 순행 시간 역행 시간 공존 되는 시점 영화 한번 보고 완벽하게 이해 하기는 어려운 부분 있지만 그게 완벽 이해 안된다고 해서 영화 감상 하는데 크게 불편 느낄 정도 아님',
 '일단 음악 점 먹고 들어간다 코로나 영화관 본게 후회된다 영화 세번 보다 보면 절묘한 설정 놀라게 된다 비현실적 부분 눈 띄게 되지만 그것 시간 역행 일어나는 상황 전혀 이상하지 않게 보인다 한가지 특이한 점 주인공 존 연기 좀 떠보였다 주인공 어딘 현실 발 붙이 하는 눈빛 행동 액션영화 라면 어울릴 둔한 전투씬 계속 눈 들어왔다 것 감독 의도 연기 그 비현실적 인물 비 전투 인간 일부러 현실 부유하는 느낌 주기 위해 그런 연기 원했고 존 데이비드 의도 해서 연기 했다면 대박 천재 들임 어쨋 영

In [56]:
# 바이너리 파일로 저장
with open('c:/data/contents_token2.pkl', 'wb') as file:
    pickle.dump(contents_token2, file)

In [67]:
tenet_daum = DataFrame({'sentens': contents_token2, 'class': daum_movie.feeling})
tenet_daum

,sentens,class
0,나 당할 순 없다,긍정
1,유명해져라 그럼 우매 대중 니 똥 싸도 박수 쳐줄것이다,부정
2,대체 내용 뭘 말 하려고 하는지 최 모르겄다 놀란 감독 골수팬 테넷 옹호 할 래야 ...,부정
3,과거 가서 췌장암 조기 발견 하면 끝 복잡하게 할 필요 있었나,중립
4,어렵다,중립
...,...,...
1024,평점 알바 많네,긍정
1025,곳곳 남겨진 흔적,긍정
1026,인셉션 인터스텔라 기대하고 보면 실망 한번 보고 재밌다고 하면 허세 충 내용 어렵고...,중립
1027,인셉션 덩케르크 다크나이트 재밌게 봤는데 이건 진짜 재미없다,긍정


In [70]:
# 바이너리 파일로 저장
with open('c:/data/tenet_naive_daum.pkl', 'wb') as file:
    pickle.dump(tenet_daum, file)

In [ ]:
---------------------------------------------------------
나이브베이즈 이론 적용

In [19]:
# 나이브베이즈 이론 적용하여 확인하는 과정

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [86]:
# 바이너리파일 불러오기
import pickle
with open('c:/data/tenet_cv.pkl', 'rb') as file:
    cv = pickle.load(file)
with open('c:/data/tenet_naive_daum.pkl', 'rb') as file:
    tenet_daum = pickle.load(file)
with open('c:/data/tenet_x_train.pkl', 'rb') as file:
    x_train = pickle.load(file)
with open('c:/data/tenet_y_train.pkl', 'rb') as file:
    y_train = pickle.load(file)

In [88]:
cv = CountVectorizer(ngram_range=(1,1))
x_train_cv = cv.fit_transform(x_train) 
x_train_cv

<5964x13246 sparse matrix of type '<class 'numpy.int64'>'
	with 72531 stored elements in Compressed Sparse Row format>

In [89]:
cv.vocabulary_

{'시간역행리': 7004,
 '스르': 6916,
 '는걸': 2457,
 '이해안감': 9396,
 '힘들다': 13242,
 '이해': 9389,
 '하기': 12216,
 '어려운': 8192,
 '종잡': 10425,
 '없는': 8362,
 '쾌감': 11595,
 '별로': 5428,
 '차원': 11152,
 '이동': 9220,
 '초대형': 11341,
 '세트': 6734,
 '실제': 7204,
 '보잉': 5719,
 '이니': 9215,
 '특수': 11850,
 '효과': 13119,
 '하는': 12243,
 '쓰잘': 7443,
 '만들고': 4098,
 '난해한': 1771,
 '복잡한': 5749,
 '만들면': 4114,
 '된다는': 3261,
 '생각': 6566,
 '가지': 133,
 '영화': 8625,
 '한다': 12448,
 '평점': 11989,
 '속지': 6780,
 '말라': 4304,
 '한심한': 12474,
 '작품': 9878,
 '전혀': 10185,
 '없다': 8373,
 '시간': 6989,
 '거슬러': 508,
 '올라가': 8760,
 '액션': 8084,
 '싸며': 7340,
 '지구': 10731,
 '위기': 9045,
 '해내는': 12617,
 '장면': 9989,
 '뭐지': 4920,
 '건가': 535,
 '감독': 244,
 '위해': 9058,
 '무려': 4801,
 '구상': 912,
 '아이디어': 7639,
 '좋음': 10431,
 '주지': 10516,
 '마라': 3948,
 '그랬다가는': 998,
 '관객': 820,
 '곤란하게': 748,
 '할테니까': 12573,
 '지금': 10734,
 '혹시': 13005,
 '보려고': 5597,
 '자가': 9771,
 '있다면': 9679,
 '아예': 7633,
 '다른': 2574,
 '티브': 11877,
 '봐라': 5857,
 '이상한': 9312,
 '메멘토': 4615,
 '확장': 1

In [79]:
tenet_daum.sentens

0                                               나 당할 순 없다
1                          유명해져라 그럼 우매 대중 니 똥 싸도 박수 쳐줄것이다
2       대체 내용 뭘 말 하려고 하는지 최 모르겄다 놀란 감독 골수팬 테넷 옹호 할 래야 ...
3                      과거 가서 췌장암 조기 발견 하면 끝 복잡하게 할 필요 있었나
4                                                     어렵다
                              ...                        
1024                                             평점 알바 많네
1025                                            곳곳 남겨진 흔적
1026    인셉션 인터스텔라 기대하고 보면 실망 한번 보고 재밌다고 하면 허세 충 내용 어렵고...
1027                    인셉션 덩케르크 다크나이트 재밌게 봤는데 이건 진짜 재미없다
1028    재밌게 봤음 사실 이런 영화 보면 볼 수록 더 이해 가지는 법 ! 평점 낮은 이유 ...
Name: sentens, Length: 1029, dtype: object

In [90]:
contents_token2_cv = cv.transform(tenet_daum.sentens)
contents_token2_cv

In [92]:
# 나이브베이즈 이론 적용하기

nb = MultinomialNB()
nb.fit(x_train_cv, y_train)

MultinomialNB()

In [94]:
daum_predict = nb.predict(contents_token2_cv)

In [ ]:
#------------------------------------------------
# 여기부터 결과

In [95]:
accuracy_score(daum_movie.feeling, daum_predict)

0.5354713313896987

In [96]:
pd.crosstab(daum_movie.feeling, daum_predict)

col_0,긍정,부정,중립
feeling,,,
긍정,499,59,14
부정,202,45,12
중립,174,17,7


In [97]:
print(classification_report(daum_movie.feeling, daum_predict))

              precision    recall  f1-score   support

          긍정       0.57      0.87      0.69       572
          부정       0.37      0.17      0.24       259
          중립       0.21      0.04      0.06       198

    accuracy                           0.54      1029
   macro avg       0.38      0.36      0.33      1029
weighted avg       0.45      0.54      0.45      1029

